In [7]:
!pip install datasets
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("ai4privacy/pii-masking-400k")

Generating dataset pii-masking-400k (C:/Users/Chulpan/.cache/huggingface/datasets/ai4privacy___pii-masking-400k/default/0.0.0/6e80a5852d659186493197b64790257bb60eea22)
Checksum Computation took 0.0 min
Generating train split
Generating train split: 100%|██████████| 325517/325517 [00:04<00:00, 77836.70 examples/s]
Generating validation split
Generating validation split: 100%|██████████| 81379/81379 [00:00<00:00, 134962.66 examples/s]
All the splits matched successfully.
Dataset pii-masking-400k downloaded and prepared to C:/Users/Chulpan/.cache/huggingface/datasets/ai4privacy___pii-masking-400k/default/0.0.0/6e80a5852d659186493197b64790257bb60eea22. Subsequent calls will reuse this data.


In [1]:
!pip install pyarrow pandas

In [4]:
from datasets import load_from_disk
import pandas as pd

# Укажи ПУТЬ К ПАПКЕ, где лежит датасет, а не к самому файлу!
dataset_path = 'pii-masking-400k-validation.arrow' # Например, 'pii-masking-400k-validation'

try:
    # Загружаем весь датасет с диска
    dataset = load_from_disk(dataset_path)

    print("Датасет успешно загружен!")
    print(dataset)

    # Теперь ты можешь работать с ним. Например, посмотреть первый элемент:
    print("\nПервый пример из датасета:")
    print(dataset[0])

    # Если ты хочешь преобразовать его в привычный Pandas DataFrame для анализа:
    # Внимание: это может занять много памяти, если датасет большой!
    # df = dataset.to_pandas()
    # print("\nПервые 5 строк в формате Pandas:")
    # print(df.head())

except FileNotFoundError:
    print(f"Ошибка: Директория не найдена по пути '{dataset_path}'")
    print("Убедись, что указан путь к папке, а не к файлу .arrow")
except Exception as e:
    print(f"Произошла ошибка при загрузке датасета: {e}")

c:\Users\Chulpan\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ошибка: Директория не найдена по пути 'pii-masking-400k-validation.arrow'
Убедись, что указан путь к папке, а не к файлу .arrow


In [10]:
import csv
import json
import re
from transformers import AutoTokenizer

TOKENIZER = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

TYPE_TO_LABEL = {
    'full_name': 'NAME', 'personName': 'NAME', 'givenName': 'NAME', 'contactName': 'NAME', 'user_fullname': 'NAME',
    'email': 'EMAIL', 'userMail': 'EMAIL', 'contactMail': 'EMAIL', 'eMail': 'EMAIL', 'emailAddr': 'EMAIL', 'mailbox': 'EMAIL',
    'phone': 'TELEPHONENUM', 'contact_phone': 'TELEPHONENUM', 'mobilePhone': 'TELEPHONENUM', 'cellPhone': 'TELEPHONENUM',
    'telephone': 'TELEPHONENUM', 'contactNumber': 'TELEPHONENUM', 'mobileNumber': 'TELEPHONENUM', 'phoneNumber': 'TELEPHONENUM',
    'credit_card': 'CREDITCARDNUMBER', 'card_number': 'CREDITCARDNUMBER', 'bankCard': 'CREDITCARDNUMBER', 'cardNo': 'CREDITCARDNUMBER',
    'cardDetails': 'CREDITCARDNUMBER', 'ccInfo': 'CREDITCARDNUMBER',
    'password': 'PASSWORD', 'pass': 'PASSWORD', 'passphrase': 'PASSWORD', 'pwd': 'PASSWORD', 'loginPass': 'PASSWORD',
    'secret_key': 'PASSWORD', 'credential': 'PASSWORD', 'accessKey': 'PASSWORD',
    'address': 'ADDRESS', 'deliveryAddress': 'ADDRESS', 'home_address': 'ADDRESS', 'physicalAddress': 'ADDRESS',
    'billing_address': 'ADDRESS', 'mailingAddress': 'ADDRESS',
    'ssn': 'SSN', 'nationalID': 'SSN', 'identityNumber': 'SSN', 'socialSecurityNumber': 'SSN', 'ssNumber': 'SSN',
    'tax_id': 'SSN', 'taxIdentification': 'SSN',
    'username': 'USERNAME', 'login': 'USERNAME', 'user_id': 'USERNAME', 'accountName': 'USERNAME', 'userTag': 'USERNAME', 'handle': 'USERNAME',
    'iban': 'IBAN', 'accountID': 'IBAN', 'bankNo': 'IBAN', 'accountNo': 'IBAN', 'bankID': 'IBAN',
    'license_plate': 'PLATENUMBER',
    'timestamp': 'DATETIME',
    'client_ip': 'IP'
}

def create_bio_tags(token_offsets, pii_spans_relative):
    tags = ['O'] * len(token_offsets)
    if not pii_spans_relative: return tags
    
    for pii in sorted(pii_spans_relative, key=lambda x: x['start']):
        pii_start, pii_end = pii['start'], pii['end']
        pii_type = TYPE_TO_LABEL.get(pii['type'], 'MISC')
        in_entity = False
        for i, (token_start, token_end) in enumerate(token_offsets):
            if token_start < pii_end and token_end > pii_start:
                if tags[i] == 'O':
                    if not in_entity:
                        tags[i] = f'B-{pii_type}'
                        in_entity = True
                    else:
                        tags[i] = f'I-{pii_type}'
    return tags

def mask_text(text, pii_spans_relative):
    if not pii_spans_relative: return text
    
    masked_text = list(text)
    type_counts = {}
    for pii in sorted(pii_spans_relative, key=lambda x: x['start'], reverse=True):
        label = TYPE_TO_LABEL.get(pii['type'], 'MISC')
        type_counts[label] = type_counts.get(label, 0) + 1
        placeholder = f"[{label}_{type_counts[label]}]"
        start, end = pii['start'], pii['end']
        masked_text[start:end] = list(placeholder)
    return "".join(masked_text)

def find_individual_logs(log_block, format_type):
    """Находит отдельные логи внутри большого блока текста."""
    if format_type == 'xml':
        # Находим все, что заключено в <log>...</log>
        return [match for match in re.finditer(r"<log>.*?</log>", log_block, re.DOTALL)]
    else:
        # Для JSON, CSV и text-форматов просто делим по строкам
        return [match for match in re.finditer(r".+", log_block)]

def parse_special_csv(file_content):
    """Парсер, устойчивый к переносам строк внутри полей."""
    metadata_pattern = re.compile(r',\s*(\d+)\s*,\s*(\d+)\s*,(.*?),\s*(\w+)\s*$', re.MULTILINE)
    header, *rest_of_file = file_content.split('\n', 1)
    content = rest_of_file[0] if rest_of_file else ""
    
    matches = list(metadata_pattern.finditer(content))
    last_pos = 0
    for match in matches:
        log_text_end = match.start()
        log_text_block = content[last_pos:log_text_end].strip()
        
        if log_text_block.startswith('"') and log_text_block.endswith('"'):
            log_text_block = log_text_block[1:-1].replace('""', '"')
        
        _, _, pii_info_str, format_type = match.groups()
        if pii_info_str.startswith('"') and pii_info_str.endswith('"'):
             pii_info_str = pii_info_str[1:-1].replace('""', '"')
        
        yield {'log_block': log_text_block, 'pii_info_str': pii_info_str, 'format_type': format_type}
        last_pos = match.end()

def process_logs_to_piiranha_format(input_csv_path, output_jsonl_path):
    print(f"Начинаю обработку файла: {input_csv_path}")
    total_records_written = 0
    
    with open(input_csv_path, 'r', encoding='utf-8') as infile, \
         open(output_jsonl_path, 'w', encoding='utf-8') as outfile:
        
        file_content = infile.read()
        
        for block_idx, row in enumerate(parse_special_csv(file_content)):
            log_block = row['log_block']
            format_type = row['format_type']
            
            try:
                pii_info_block = json.loads(row['pii_info_str']) if row['pii_info_str'] else []
            except json.JSONDecodeError:
                pii_info_block = []

            # --- Вложенный цикл: обрабатываем каждый лог внутри блока ---
            for log_match in find_individual_logs(log_block, format_type):
                log_text = log_match.group(0)
                log_start_pos = log_match.start()
                
                # Фильтруем и пересчитываем координаты PII для этого конкретного лога
                pii_info_relative = []
                for pii in pii_info_block:
                    if pii['start'] >= log_start_pos and pii['end'] <= log_match.end():
                        pii_info_relative.append({
                            **pii,
                            'start': pii['start'] - log_start_pos,
                            'end': pii['end'] - log_start_pos
                        })
                
                # Стандартная обработка
                tokenized = TOKENIZER(log_text, return_offsets_mapping=True, add_special_tokens=False)
                bio_tags = create_bio_tags(tokenized['offset_mapping'], pii_info_relative)
                masked = mask_text(log_text, pii_info_relative)

                output_record = {
                    "source_text": log_text,
                    "masked_text": masked,
                    "tokens": tokenized.tokens(),
                    "ner_labels": bio_tags,
                    "labels": [{**pii, "label": TYPE_TO_LABEL.get(pii['type'], 'MISC'), "label_index": i+1} 
                               for i, pii in enumerate(pii_info_relative)],
                    "locale": "RU", "language": "ru", "split": "train"
                }
                
                outfile.write(json.dumps(output_record, ensure_ascii=False) + '\n')
                total_records_written += 1
    
    print("-" * 50)
    print("Обработка завершена.")
    print(f"Сохранено в файл: {output_jsonl_path}")
    print(f"Всего создано записей: {total_records_written}")


if __name__ == "__main__":
    input_file = "synthetic_logs.csv" 
    output_file = "piiranha_formatted_logs.jsonl"
    process_logs_to_piiranha_format(input_file, output_file)

Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors


Начинаю обработку файла: synthetic_logs.csv
--------------------------------------------------
Обработка завершена.
Сохранено в файл: piiranha_formatted_logs.jsonl
Всего создано записей: 2417


In [1]:
from datasets import load_dataset
import pandas as pd

# Указываем имя датасета, как на Hugging Face Hub.
# Библиотека сама найдет его в твоем кэше и правильно соберет.
dataset_name = "ai4privacy/pii-masking-400k"

try:
    # Загружаем датасет (он будет взят из кэша, скачиваться не будет)
    dataset = load_dataset(dataset_name)

    print("Датасет успешно загружен из кэша!")
    print(dataset)

    # Выбираем нужную часть (например, validation)
    validation_data = dataset['train'] # В этом датасете нет 'validation', только 'train' и 'test'

    # Посмотрим на первый пример
    print("\n--- Первый элемент датасета ---")
    print(validation_data[0])

    # Для удобного анализа можно преобразовать часть данных в Pandas DataFrame
    # Внимание: не делай это для всего датасета сразу, 400к строк могут занять всю память!
    # Возьмем первые 1000 строк.
    df = validation_data.select(range(1000)).to_pandas()

    print("\n--- Первые 5 строк в виде таблицы Pandas ---")
    print(df.head())

except Exception as e:
    print(f"Произошла ошибка: {e}")
    print("Убедись, что имя датасета верное и кэш не поврежден.")

c:\Users\Chulpan\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Датасет успешно загружен из кэша!
DatasetDict({
    train: Dataset({
        features: ['source_text', 'locale', 'language', 'split', 'privacy_mask', 'uid', 'masked_text', 'mbert_tokens', 'mbert_token_classes'],
        num_rows: 325517
    })
    validation: Dataset({
        features: ['source_text', 'locale', 'language', 'split', 'privacy_mask', 'uid', 'masked_text', 'mbert_tokens', 'mbert_token_classes'],
        num_rows: 81379
    })
})

--- Первый элемент датасета ---
{'source_text': '<p>My child faozzsd379223 (DOB: May/58) will undergo treatment with Dr. faozzsd379223, office at Hill Road. Our ZIP code is 28170-6392. Consult policy M.UE.227995. Contact number: 0070.606.322.6244. Handle transactions with 6225427220412963. Queries? Email: faozzsd379223@outlook.com.</p>', 'locale': 'US', 'language': 'en', 'split': 'train', 'privacy_mask': [{'label': 'USERNAME', 'start': 12, 'end': 25, 'value': 'faozzsd379223', 'label_index': 2}, {'label': 'DATEOFBIRTH', 'start': 32, 'end': 38, 'va